In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Metdata

In [2]:
yah = pd.read_csv("../data/yahoo.csv")
cap = pd.read_csv("../data/capfriendly.csv")

### Master List

In [3]:
master = pd.merge(yah, cap, how="left", on="name")

In [4]:
master.head()

,name,team,position,pick,age,salary
0,Connor McDavid,Edm,C,1.7,23.0,14000000.0
1,Nathan MacKinnon,Col,C,2.6,25.0,6150000.0
2,Leon Draisaitl,Edm,"C,LW",3.7,25.0,9000000.0
3,Artemi Panarin,NYR,LW,5.3,29.0,13000000.0
4,Alex Ovechkin,Was,LW,5.6,35.0,10000000.0


### Multiple Positions

In [5]:
multi = (
    master
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

### Projection Data

In [6]:
cbs = pd.read_csv("../data/cbs.csv")
dfo = pd.read_csv("../data/dailyfaceoff.csv")
proj = pd.concat([cbs, dfo])

### Goalies

In [7]:
goalies = proj[proj["position"] == "G"].copy()

GOALIE_CATEGORIES = ["wins", "goals_against_average","saves", "shutouts"]

goalies['goals_against_average'] = -goalies['goals_against_average']

goalies = (
    goalies
    [["name"] + GOALIE_CATEGORIES]
    .groupby("name")
    .mean()
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

goalies["wins"] *= 1/1
goalies["goals_against_average"] *= 3/4
goalies["saves"] *= 3/4
goalies["shutouts"] *= 1/2

goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= 1 + 3/4 + 3/4 + 1/2
goalies["rollup"] *= 100

goalies = goalies.reset_index()

goalies = goalies[["name", "rollup"]]

### Skaters

In [8]:
skaters = proj[proj["position"] != "G"].copy()

SKATER_CATEGORIES = ['goals', 'assists', 'plus_minus', 'powerplay_points', 'shots_on_goal', 'hits', 'blocks']

skaters = skaters[["name"] + SKATER_CATEGORIES]

skaters = (
    skaters
    [["name"] + SKATER_CATEGORIES]
    .groupby("name")
    .mean()
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

skaters.sort_values("name")

skaters['goals'] *= 7/8
skaters['assists'] *= 7/8
skaters['plus_minus'] *= 3/4
skaters['powerplay_points'] *= 3/4
skaters['shots_on_goal'] *= 1
skaters['hits'] *= 1
skaters['blocks'] *= 1

skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= 7/8 + 7/8 + 3/4 +3/4 + 1 + 1 + 1
skaters["rollup"] *= 100

skaters = skaters.reset_index()

skaters = skaters[["name", "rollup"]]

### Merge

In [9]:
rollup = pd.concat([skaters, goalies])
df = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
df.head()

,name,position,rollup
7,Andrei Vasilevskiy,G,93.309038
83,Philipp Grubauer,G,82.063000
35,Tuukka Rask,G,81.849230
39,Robin Lehner,G,81.442042
38,Carter Hart,G,80.446691


### VORP

In [10]:
# pool particulars
pool_size = 12
starters = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

df["vorp"] = df["rollup"]

for position, slots in starters.items():
    replacement = (
        df[df['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * pool_size)
        ['vorp']
        .mean()
    )
    df.loc[df['position'] == position, 'vorp'] = df['vorp'] - replacement
    
df = df.sort_values("vorp", ascending=False)
df = df.groupby("name").head(1)[["name", "rollup", "vorp"]]
df['rank'] = df['vorp'].rank(method='average', ascending=False)

### Draft

In [11]:
draft = pd.merge(master, df, how="left", on="name")
draft["arbitrage"] = draft["pick"] - draft["rank"]
draft['round'] = (draft['pick'] // pool_size) + 1

draft = draft[['name', 'team', 'position', 'age', 'rollup', 'vorp', "round", 'pick', 'rank', 'arbitrage']]
draft = draft.sort_values("rank")
draft.head()

,name,team,position,age,rollup,vorp,round,pick,rank,arbitrage
6,Andrei Vasilevskiy,TB,G,26.0,93.309038,20.816607,1.0,7.6,1.0,6.6
1,Nathan MacKinnon,Col,C,25.0,63.015053,15.881379,1.0,2.6,2.0,0.6
29,Roman Josi,Nsh,D,30.0,53.229728,13.067851,3.0,32.8,3.0,29.8
0,Connor McDavid,Edm,C,23.0,58.994859,11.861184,1.0,1.7,4.0,-2.3
4,Alex Ovechkin,Was,LW,35.0,57.153766,11.590759,1.0,5.6,5.0,0.6


In [12]:
draft.to_csv("../data/draft.csv", index=False)